In [1]:
import requests

url = "https://www.dunkest.com/en/euroleague/stats/teams/defense-vs-position/season/2024-2025?season_id=17&stats_id=4&position_id=1&sort_by=all&sort_order=desc"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

print("Status Code:", response.status_code)
print("Response Length:", len(response.text))  # Check if it's a valid page


Status Code: 200
Response Length: 39228


In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")

# Print the first 1000 characters to inspect
print(soup.prettify()[:1000])

# Try finding the table
table = soup.find("table")  # Check if table is found
if table:
    print("✅ Table Found!")
else:
    print("⚠️ Table Not Found!")


<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Meta tags -->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <title>
   Euroleague Defense VS Position - Season 2024/25 | Dunkest
  </title>
  <meta content="Defense VS Position Euroleague. View the average stats and fantasy points allowed by each Euroleague team by position, Season 2024-2025." name="description"/>
  <!-- Multilingual Meta Tags -->
  <link href="https://www.dunkest.com/it/eurolega/statistiche/squadre/difesa-vs-posizione/season/2024-2025" hreflang="it" rel="alternate"/>
  <link href="https://www.dunkest.com/en/euroleague/stats/teams/defense-vs-position/season/2024-2025" hreflang="en" rel="alternate"/>
  <!-- Meta Tag Canonical -->
  <link href="https://www.dunkest.com/en/euroleague/stats/teams/defense-vs-position/season/2024-2025" rel="canonical"/>
  <!-- OG Meta Tags -->
  <meta content="Euroleague Defense VS Position - Season 2024/25 | Dunkest" proper

In [3]:
print("📊 Raw Table HTML:\n", table.prettify()[:2000])  # Print first 2000 characters



📊 Raw Table HTML:
 <table class="table table--stats">
 <thead id="statsTableHead">
 </thead>
 <tbody id="statsTableBody">
 </tbody>
</table>



In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# ✅ Define positions and corresponding URL parameters
positions = {
    1: "Guard",
    2: "Forward",
    3: "Center"
}

# ✅ Set up Selenium WebDriver
options = Options()
options.headless = True  # Run in headless mode (no browser window)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ✅ Initialize an empty dictionary to store soup objects for each position
soups = {}

for position_id, position_name in positions.items():
    print(f"\n🔍 Loading webpage for {position_name}...\n")

    # ✅ Load the webpage
    url = f"https://www.dunkest.com/en/euroleague/stats/teams/defense-vs-position/season/2024-2025?season_id=17&stats_id=4&position_id={position_id}&sort_by=all&sort_order=desc"
    driver.get(url)

    # ✅ Wait for JavaScript to load
    time.sleep(5)

    # ✅ Store the parsed page source in the dictionary
    from bs4 import BeautifulSoup
    soups[position_name] = BeautifulSoup(driver.page_source, "html.parser")

print("✅ Webpages loaded successfully!")



🔍 Loading webpage for Guard...


🔍 Loading webpage for Forward...


🔍 Loading webpage for Center...

✅ Webpages loaded successfully!


In [5]:
# ✅ Initialize a dictionary to store extracted tables
tables = {}

for position_name, soup in soups.items():
    # ✅ Locate the table
    table = soup.find("table", class_="table table--stats")

    if table:
        tables[position_name] = table
        print(f"✅ Table found for {position_name}!")
    else:
        print(f"❌ Table NOT found for {position_name}. Check website structure.")


✅ Table found for Guard!
✅ Table found for Forward!
✅ Table found for Center!


In [6]:
# ✅ Extract headers from the tables
headers = None  # Store headers once to use for all positions

for position_name, table in tables.items():
    # ✅ Extract headers if not already done
    if headers is None:
        extracted_headers = [th.text.strip() for th in table.find_all("th")]

        # ✅ Keep only relevant columns (ignore ranking numbers 1-18)
        valid_columns = ["#", "Team", "L3", "L5", "L10", "ALL"]
        headers = [col for col in extracted_headers if col in valid_columns]
        headers.append("Position")  # ✅ Add "Position" column

    print(f"📊 Headers for {position_name}: {headers}")


📊 Headers for Guard: ['#', 'Team', 'L3', 'L5', 'L10', 'ALL', 'Position']
📊 Headers for Forward: ['#', 'Team', 'L3', 'L5', 'L10', 'ALL', 'Position']
📊 Headers for Center: ['#', 'Team', 'L3', 'L5', 'L10', 'ALL', 'Position']


In [7]:
# ✅ Initialize a dictionary to store rows for each position
position_rows = {}

for position_name, table in tables.items():
    rows = []
    
    for tr in table.find_all("tr"):  # ✅ Extract rows
        cols = [td.text.strip() for td in tr.find_all("td")]

  
        cols.append(position_name)  # ✅ Add position
        rows.append(cols)

    position_rows[position_name] = rows

    print(f"📊 Sample data for {position_name}: {rows[:3]}")  # Print first 3 rows


📊 Sample data for Guard: [['Guard'], ['EA7 Emporio Armani Milan', '14.3', '13.9', '11.6', '12.7', 'Guard'], ['Virtus Segafredo Bologna', '14.4', '13.5', '12.8', '12.5', 'Guard']]
📊 Sample data for Forward: [['Forward'], ['FC Bayern Munich', '10.0', '9.0', '12.1', '12.4', 'Forward'], ['Maccabi Playtika Tel Aviv', '11.5', '11.9', '12.8', '12.1', 'Forward']]
📊 Sample data for Center: [['Center'], ['Crvena Zvezda Meridianbet Belgrade', '11.0', '10.0', '12.6', '12.1', 'Center'], ['Paris Basketball', '10.0', '9.3', '11.1', '12.0', 'Center']]


In [8]:
import pandas as pd

# ✅ Define correct headers
headers = ["Team", "L3", "L5", "L10", "ALL", "Position"]

# ✅ Combine all position data into a single list
all_rows = []
for position, rows in position_rows.items():
    all_rows.extend(rows)

# ✅ Convert to Pandas DataFrame
df = pd.DataFrame(all_rows, columns=headers)

# ✅ Display DataFrame
print(df)


                                  Team    L3    L5   L10   ALL Position
0                                Guard  None  None  None  None     None
1             EA7 Emporio Armani Milan  14.3  13.9  11.6  12.7    Guard
2             Virtus Segafredo Bologna  14.4  13.5  12.8  12.5    Guard
3                         FC Barcelona  13.0  12.7  12.2  12.4    Guard
4             Baskonia Vitoria-Gasteiz  12.7  13.3  13.0  12.3    Guard
5                          Real Madrid  10.6  10.9  11.6  12.1    Guard
6            Maccabi Playtika Tel Aviv  11.1  10.1  11.8  12.0    Guard
7                          ALBA Berlin   6.5  11.2  11.4  11.9    Guard
8              LDLC ASVEL Villeurbanne  11.2  12.1  11.3  11.8    Guard
9             Fenerbahce Beko Istanbul  10.6  10.8  11.6  11.7    Guard
10                    Paris Basketball  12.9  11.9  12.5  11.7    Guard
11                    FC Bayern Munich  11.3  12.0  12.4  11.6    Guard
12                     Zalgiris Kaunas   9.3  13.1  12.3  11.6  

In [16]:
# ✅ Remove rows where any column contains None or NaN values
defense_vs_position_df = df.dropna()

# ✅ Reset index for cleaner display
defense_vs_position_df = df.reset_index(drop=True)

# ✅ Display cleaned DataFrame
print(defense_vs_position_df)


                                  Team    L3    L5   L10   ALL Position
0             EA7 Emporio Armani Milan  14.3  13.9  11.6  12.7    Guard
1             Virtus Segafredo Bologna  14.4  13.5  12.8  12.5    Guard
2                         FC Barcelona  13.0  12.7  12.2  12.4    Guard
3             Baskonia Vitoria-Gasteiz  12.7  13.3  13.0  12.3    Guard
4                          Real Madrid  10.6  10.9  11.6  12.1    Guard
5            Maccabi Playtika Tel Aviv  11.1  10.1  11.8  12.0    Guard
6                          ALBA Berlin   6.5  11.2  11.4  11.9    Guard
7              LDLC ASVEL Villeurbanne  11.2  12.1  11.3  11.8    Guard
8             Fenerbahce Beko Istanbul  10.6  10.8  11.6  11.7    Guard
9                     Paris Basketball  12.9  11.9  12.5  11.7    Guard
10                    FC Bayern Munich  11.3  12.0  12.4  11.6    Guard
11                     Zalgiris Kaunas   9.3  13.1  12.3  11.6    Guard
12                  Olympiacos Piraeus  12.3  12.4  11.0  11.5  

In [17]:
defense_vs_position_df.attrs["dataset_name"] = "EuroLeague_Defense_vs_Position"
defense_vs_position_df.attrs["description"] = "EuroLeague defensive statistics against different positions over multiple timeframes."
defense_vs_position_df.attrs["source"] = "https://www.dunkest.com/en/euroleague/stats/teams/defense-vs-position/"
defense_vs_position_df.attrs["column_info"] = {
    "Team": "Team name",
    "L3": "Defense vs this position in the last 3 games",
    "L5": "Defense vs this position in the last 5 games",
    "L10": "Defense vs this position in the last 10 games",
    "ALL": "Overall defense vs this position",
    "Position": "Player position (Guard, Forward, Center)"
}


In [11]:


options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://www.rezultati.com/kosarka/europa/euroleague/raspored/"
driver.get(url)



# ✅ Wait for JavaScript to load
time.sleep(5)  # Adjust if needed

# ✅ Parse the fully loaded page source
soup = BeautifulSoup(driver.page_source, "html.parser")

html_source = driver.page_source

# ✅ Close Selenium (we now use only BeautifulSoup)
driver.quit()

print(html_source[:500])

<html lang="hr"><head>
        <meta charset="utf-8">
        <title>Euroleague raspored - Košarka/Europa</title>
        <script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-999393459&amp;cx=c&amp;gtm=45He52q0v71314122za200&amp;tag_exp=101732282~101732284~102067808~102482433~102539968~102558064~102587591~102605417~102640600~102643510~102717422~102732003"></script><script type="text/javascript" async="" src="https://unpkg.com/web-vitals/dist/web-vi


In [64]:


# ✅ Find all match events
matches = []
for match_row in soup.find_all("div", class_="event__match"):
    try:
        date = match_row.find_previous("div", class_="event__time").text.strip()
        home_team = match_row.find("div", class_="event__participant--home").text.strip()
        away_team = match_row.find("div", class_="event__participant--away").text.strip()

        matches.append([date, home_team, away_team])

    except AttributeError:
        print("⚠️ Skipping a row due to missing data.")

# ✅ Convert to Pandas DataFrame
schedule_df = pd.DataFrame(matches, columns=["Date", "Home Team", "Away Team"])

import pandas as pd
from datetime import datetime

# ✅ Assume `schedule_df` is already loaded
# Example: schedule_df = pd.read_csv("your_schedule_file.csv") if loading from CSV

# ✅ Function to convert date format
def convert_date(date_str):
    """Convert 'dd.mm. HH:MM' to 'YYYY-MM-DD HH:MM' format (assuming 2025 as year)."""
    try:
        fixed_date_str = f"{date_str} 2025"  # Append year manually
        dt = datetime.strptime(fixed_date_str, "%d.%m. %H:%M %Y")  # Parse with appended year
        return dt  # Return as datetime object
    except ValueError as e:
        print(f"⚠️ Error parsing date '{date_str}': {e}")
        return pd.NaT  # Return NaT (Not a Time) for failed conversions

# ✅ Apply conversion to DataFrame
schedule_df["Date"] = schedule_df["Date"].apply(convert_date)

# ✅ Ensure `Date` column is in datetime format
schedule_df["Date"] = pd.to_datetime(schedule_df["Date"])




⚠️ Skipping a row due to missing data.


In [65]:
print(schedule_df[:10])

                 Date          Home Team          Away Team
0 2025-03-05 20:30:00           Baskonia    Zalgiris Kaunas
1 2025-03-06 20:30:00             Bayern      Crvena zvezda
2 2025-03-06 20:30:00             Milano         Fenerbahce
3 2025-03-06 20:30:00      Panathinaikos        Real Madrid
4 2025-03-06 20:45:00       Anadolu Efes     Virtus Bologna
5 2025-03-07 18:30:00  Lyon-Villeurbanne   Maccabi Tel Aviv
6 2025-03-07 20:00:00         Olympiakos           Partizan
7 2025-03-07 20:15:00        Alba Berlin          Barcelona
8 2025-03-07 20:30:00         Fenerbahce  Lyon-Villeurbanne
9 2025-03-13 18:45:00      Crvena zvezda             Milano


In [66]:
# ✅ Define metadata for the EuroLeague schedule DataFrame
schedule_df.attrs = {
    "dataset_name": "EuroLeague_Schedule",
    "description": "This dataset contains the EuroLeague basketball schedule, including match dates, home teams, and away teams.",
    "source": "https://www.rezultati.com/kosarka/europa/euroleague/raspored/",
    "last_updated": pd.Timestamp.now(),  # ✅ Automatically sets the current timestamp
    "columns": {
        "Date": "The date of the match (DD.MM. format)",
        "Home Team": "The team playing at home",
        "Away Team": "The visiting team",
    },
    "usage": "Use this dataset for retrieving match schedules and predicting team performance.",
}

# ✅ Print the metadata to verify
print(schedule_df.attrs)


{'dataset_name': 'EuroLeague_Schedule', 'description': 'This dataset contains the EuroLeague basketball schedule, including match dates, home teams, and away teams.', 'source': 'https://www.rezultati.com/kosarka/europa/euroleague/raspored/', 'last_updated': Timestamp('2025-03-04 13:44:01.744265'), 'columns': {'Date': 'The date of the match (DD.MM. format)', 'Home Team': 'The team playing at home', 'Away Team': 'The visiting team'}, 'usage': 'Use this dataset for retrieving match schedules and predicting team performance.'}


In [68]:
schedule_df["Date"] = schedule_df["Date"].dt.strftime("%Y-%m-%d %H:%M")


from langchain.schema import Document
import json

# ✅ Convert schedule DataFrame to documents
schedule_documents = [
    Document(
        page_content=f"{row['Home Team']} vs {row['Away Team']} on {row['Date']}",
        metadata={
            "type": "schedule",
            "date": row["Date"],
            "home_team": row["Home Team"],
            "away_team": row["Away Team"],
        }
    )
    for _, row in schedule_df.iterrows()
]

# ✅ Convert defense stats DataFrame to documents
defense_documents = [
    Document(
        page_content=f"{row['Team']} allows {row['ALL']} fantasy points to {row['Position']}",
        metadata={
            "type": "defense_stats",
            "team": row["Team"],
            "L3": row["L3"],
            "L5": row["L5"],
            "L10": row["L10"],
            "ALL": row["ALL"],
            "position": row["Position"],
        }
    )
    for _, row in defense_vs_position_df.iterrows()
]

# ✅ Combine both datasets into one list
all_documents = schedule_documents + defense_documents

# ✅ Store metadata from .attrs separately
metadata_info = {
    "schedule_metadata": {
        "dataset_name": schedule_df.attrs.get("dataset_name", "EuroLeague_Schedule"),
        "description": schedule_df.attrs.get("description", "Schedule for EuroLeague games"),
        "source": schedule_df.attrs.get("source", "https://www.rezultati.com/kosarka/europa/euroleague/raspored/"),
        "columns": list(schedule_df.columns),
    },
    "defense_metadata": {
        "dataset_name": defense_vs_position_df.attrs.get("dataset_name", "EuroLeague_Defense_vs_Position"),
        "description": defense_vs_position_df.attrs.get("description", "Defensive stats against different positions"),
        "source": defense_vs_position_df.attrs.get("source", "https://www.dunkest.com/en/euroleague/stats/teams/defense-vs-position/"),
        "columns": list(defense_vs_position_df.columns),
    },
}

# ✅ Convert documents to JSON format
documents_json = [
    {"content": doc.page_content, "metadata": doc.metadata} for doc in all_documents
]

# ✅ Combine documents and metadata into a single JSON file
final_data = {
    "documents": documents_json,
    "metadata": metadata_info
}

# ✅ Save to a single JSON file
with open("rag_documents.json", "w", encoding="utf-8") as f:
    json.dump(final_data, f, indent=4)

print(f"✅ {len(all_documents)} documents saved to 'rag_documents.json' with metadata!")


✅ 116 documents saved to 'rag_documents.json' with metadata!


In [69]:
with open("rag_documents.json", "r", encoding="utf-8") as f:
    data = json.load(f)

documents = [
    Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in data["documents"]
]

print(f"Loaded {len(documents)} documents")
print("📌 Sample document:", documents[0])

Loaded 116 documents
📌 Sample document: page_content='Baskonia vs Zalgiris Kaunas on 2025-03-05 20:30' metadata={'type': 'schedule', 'date': '2025-03-05 20:30', 'home_team': 'Baskonia', 'away_team': 'Zalgiris Kaunas'}


✅ JSON file updated successfully!


In [70]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ✅ Define a chunking strategy (split at 200 characters, with 20 overlap)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=20, separators=[" ", "\n"]
)

# ✅ Apply chunking while preserving metadata
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        chunked_documents.append(Document(page_content=chunk, metadata=doc.metadata))  # Preserve metadata

# ✅ Check the results
print(f"Total chunked documents: {len(chunked_documents)}")
print("📌 Sample chunked document:", chunked_documents[0])


Total chunked documents: 116
📌 Sample chunked document: page_content='Baskonia vs Zalgiris Kaunas on 2025-03-05 20:30' metadata={'type': 'schedule', 'date': '2025-03-05 20:30', 'home_team': 'Baskonia', 'away_team': 'Zalgiris Kaunas'}


In [71]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# ✅ Initialize embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-large-en-v1.5",
    model_kwargs={"trust_remote_code": True}  # 👈 Add this line
)
# ✅ Test the embedding model
test_text = "Real Madrid vs Barcelona on 15th March"
vector = embedding_model.embed_query(test_text)


In [72]:
from langchain_community.vectorstores import Chroma

# ✅ Initialize ChromaDB (local storage)
vectorstore = Chroma(collection_name="euroleague_rag", embedding_function=embedding_model)


In [73]:
# ✅ Store documents in ChromaDB
vectorstore.add_documents(chunked_documents)

print(f"✅ {len(chunked_documents)} documents stored in ChromaDB!")


✅ 116 documents stored in ChromaDB!


In [95]:
# ✅ Initialize retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 100})  # Top 5 results

# ✅ Example query
query = "Which team has the best defence against centers?"
retrieved_docs = retriever.get_relevant_documents(query)

# ✅ Remove duplicates based on `page_content`
unique_docs = []
seen_texts = set()

for doc in retrieved_docs:
    if doc.page_content not in seen_texts:
        unique_docs.append(doc)
        seen_texts.add(doc.page_content)

retrieved_docs = unique_docs  # Use only unique documents
print(f"🔍 Unique relevant documents: {len(retrieved_docs)}")


print(f"🔍 Found {len(retrieved_docs)} relevant documents")


def format_doc(doc):
    """Dynamically format retrieved documents with all metadata."""
    metadata_info = ", ".join(f"{k}: {v}" for k, v in doc.metadata.items())
    return f"Match Info: {doc.page_content.strip()} | {metadata_info}"

retrieved_text = "\n".join(format_doc(doc) for doc in retrieved_docs)
print(retrieved_text)
  
    

🔍 Unique relevant documents: 50
🔍 Found 50 relevant documents
Match Info: Paris Basketball allows 12.0 fantasy points to Center | ALL: 12.0, L10: 11.1, L3: 10.0, L5: 9.3, position: Center, team: Paris Basketball, type: defense_stats
Match Info: Zalgiris Kaunas allows 9.3 fantasy points to Center | ALL: 9.3, L10: 9.7, L3: 7.5, L5: 10.8, position: Center, team: Zalgiris Kaunas, type: defense_stats
Match Info: Maccabi Playtika Tel Aviv allows 10.5 fantasy points to Center | ALL: 10.5, L10: 8.0, L3: 12.3, L5: 11.5, position: Center, team: Maccabi Playtika Tel Aviv, type: defense_stats
Match Info: FC Barcelona allows 9.1 fantasy points to Center | ALL: 9.1, L10: 10.0, L3: 4.5, L5: 10.5, position: Center, team: FC Barcelona, type: defense_stats
Match Info: Olympiacos Piraeus allows 9.9 fantasy points to Center | ALL: 9.9, L10: 10.2, L3: 12.0, L5: 11.4, position: Center, team: Olympiacos Piraeus, type: defense_stats
Match Info: Panathinaikos AKTOR Athens allows 10.8 fantasy points to Center |

In [93]:
# Check unique documents count
unique_page_content = set(doc.page_content for doc in retrieved_docs)
print(f"Unique documents: {len(unique_page_content)} / {len(retrieved_docs)}")


Unique documents: 25 / 50


In [90]:
from langchain.chains import RetrievalQA, StuffDocumentsChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama

# ✅ Initialize LLM (Ollama)
llm = Ollama(model="llama3.2", base_url="http://localhost:11434")

# ✅ Define a prompt to structure retrieved documents
qa_prompt = PromptTemplate.from_template(
    "You are an expert in EuroLeague basketball. Use ONLY the provided documents to answer the question.\n\n"
    "Retrieved Documents:\n"
    "{context}\n\n"
    "Question: {question}\n\n"
    "Instructions:\n"
    "1. Consider ALL listed matches when answering.\n"
    
    "3. If the context does not contain enough data, reply: 'I do not have enough information.'\n"
    "Answer:"
)

# ✅ Create prompt manually
prompt = qa_prompt.format(context=retrieved_text, question=query)

# ✅ Generate response directly from LLM
response = llm(prompt)

print("🔍 Custom Response:", response)

# # ✅ Create document combining chain
# combine_documents_chain = StuffDocumentsChain(
#     llm_chain=LLMChain(llm=llm, prompt=qa_prompt),
#     document_variable_name="context"
# )

# # ✅ Create the RetrievalQA pipeline
# qa_chain = RetrievalQA(
#     retriever=retriever,
#     combine_documents_chain=combine_documents_chain
# )

# # ✅ Ask a question
# query = "Who is Real Madrid playing in the next 7 matches?"
# response = qa_chain.invoke(query)


🔍 Custom Response: Based on the provided data, Virtus Segafredo Bologna and Anadolu Efes Istanbul have the lowest average fantasy points allowed to centers, with 10.2 and 11.1, respectively.

However, considering the context, it's essential to note that a lower number doesn't necessarily mean better defense. A team might allow fewer points but still struggle to limit the opponent's scoring effectiveness.

If we consider only the lowest average fantasy points allowed to centers (which is the case here), Virtus Segafredo Bologna has the best defence against centers, with 10.2 fantasy points allowed on average.


In [92]:
query = "Which teams have the hardest match for guards and why?"

retrieved_docs = retriever.get_relevant_documents(query)
retrieved_text = "\n".join(format_doc(doc) for doc in retrieved_docs)

prompt = qa_prompt.format(context=retrieved_text, question=query)

# ✅ Generate response directly from LLM
response = llm(prompt)

print("🔍 Custom Response:", response)

🔍 Custom Response: Based on the provided data, I've analyzed the matches for each team to determine which teams have the hardest match for guards.

The top two teams with the hardest match for guards are:

1. EA7 Emporio Armani Milan:
	* Their guard has a score of 12.7 in ALL matches.
	* The difference between their guard's score and the highest guard score is significant (13.9 - 12.7 = 1.2).
	* This indicates that their guards are among the best performers, making it challenging for them to face opponents with stronger guards.
2. Paris Basketball:
	* Their center has a score of 12.0 in ALL matches.
	* Although not directly comparing guard scores, this high center score suggests that Paris Basketball might have strong overall team performance, including their guards, which could make it harder for guards from other teams to match up against them.

These two teams appear to have the hardest match for guards due to their strong overall team performance and the exceptional skills of their